In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from common import *

### Evaluate single experiment

In [2]:
checkpoint = 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_1_tr_s1_ts_s5678'
args, opts = get_opt_args_from_model_name(checkpoint, mode='latest')
model_pos, chk_filename, checkpoint = load_model(opts, args)

subset = 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678'
# args.subset_list = [subset]
train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)

# if '3DHP' in subset:
#     args.eval_part = 'whole_without_nose'
#     args.part_list = ['whole_without_nose']

configs/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_1_tr_s1_ts_s5678.yaml
True
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_1_tr_s1_ts_s5678
Loading checkpoint checkpoint/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_1_tr_s1_ts_s5678/latest_epoch.bin
Loading dataset...
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-STEP_ROT_1-TR_S1_TS_S5678
[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_3d_steprot_1.0...
[overwrite: False] ==> Loading H36M img_2d_canonical_same_z_steprot_1.0...


KeyboardInterrupt: 

In [16]:
e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)

INFO: Testing
No epoch information in the checkpoint


  0%|          | 0/323 [00:00<?, ?it/s]

100%|██████████| 323/323 [01:47<00:00,  3.01it/s]


No eval_keypoint. Use part list
Part: whole
+------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| test_name  |     Directions    |    Dir

### Evaluate and save all experiments

In [2]:
result_list = os.listdir('saved_results')
whitelist_checkpoint = []
for item in os.listdir('configs/pose3d'):
    checkpoint = item.split('.yaml')[0]
    if 'cam_no_factor' not in checkpoint: continue
    if checkpoint in blacklist_checkpoint: continue
    if checkpoint not in experiment_list:
        whitelist_checkpoint.append(checkpoint)
    else:
        subset_list = []
        with open(f'experiments/{checkpoint}.txt', 'r') as f:
            lines = f.readlines()
            for line in lines:
                subset = line.split('\n')[0]
                subset_list.append(subset)
        for subset in subset_list:
            if not os.path.exists(f'saved_results/{checkpoint}/{subset}.pkl'):
                print(f'{checkpoint} - {subset} not found')
                if checkpoint not in whitelist_checkpoint: 
                    whitelist_checkpoint.append(checkpoint)
                    
print('\nCheckpoints to evaluate:')
whitelist_checkpoint

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.5_tr_s1_ts_s5678 - 3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN not found
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.5_tr_s1_ts_s5678 - 3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6 not found
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.5_tr_s1_ts_s5678 - H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z not found
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.5_tr_s1_ts_s5678 - FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST not found
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.5_tr_s1_ts_s5678 - H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678 not found
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.5_tr_s1_ts_

['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.1_tr_s1_ts_s5678',
 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.5_tr_s1_ts_s5678',
 'MB_train_h36m_gt_cam_no_factor_steprot_1_tr_s1_ts_s5678',
 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_steprot_1_tr_s1_ts_s5678',
 'MB_train_h36m_gt_cam_no_factor_with_canonical2_steprot_1_tr_s1_ts_s5678']

In [3]:
experiment_root = f'/home/{user}/codes/MotionBERT/experiments'
for item in glob(experiment_root+'/*.txt'):
    checkpoint = os.path.basename(item).split('.txt')[0]
    if checkpoint in blacklist_checkpoint: continue
    if len(whitelist_checkpoint) > 0 and checkpoint not in whitelist_checkpoint: continue
    if 'cam_no_factor' not in checkpoint: continue
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint}'
    if not os.path.exists(save_folder): os.makedirs(save_folder)
    
    # load model
    args, opts = get_opt_args_from_model_name(checkpoint, verbose=False)
    model_pos, chk_filename, checkpoint = load_model(opts, args)
    
    subset_list = []
    with open(item, 'r') as f:
        lines = f.readlines()
        for line in lines:
            subset = line.strip()
            subset_list.append(subset)
    
    for subset in subset_list:
        save_path = os.path.join(save_folder, f'{subset}.pkl')
        if os.path.exists(save_path): 
            print(f'{save_path} exists')
            continue
        try:
            # load dataset
            args.subset_list = [subset]
            train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
            # evaluation
            e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)
            # save results
            results_dict = {'results_all': results_all, 'gts_all': gts_all, 'inputs_all': inputs_all, 'total_result_dict': total_result_dict, 'e1': e1, 'e2': e2}
            savepkl(results_dict, save_path)
        except Exception as e:
            print(f'Error: {e}')


MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.5_tr_s1_ts_s5678
Loading checkpoint checkpoint/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_steprot_0.5_tr_s1_ts_s5678/best_epoch.bin
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_same_z...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_same_z...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 245/245 [01:23<00:00,  2.93it/s]


No eval_keypoint. Use part list
Part: whole
+------------+-------------------+-------------------+
| test_name  |        Seq1       |        Seq2       |
+------------+-------------------+-------------------+
| P1 (whole) | 79.19301891488041 | 87.03035729574748 |
| P2 (whole) | 59.72370711801546 | 67.75072663785659 |
+------------+-------------------+-------------------+
Protocol #1 Error (MPJPE): 83.11168810531395 mm
Protocol #2 Error (P-MPJPE): 63.73721687793602 mm
----------------------------------------
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_same_z...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_same_z...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 6/6 [00:02<00:00,  2.20it/s]


No eval_keypoint. Use part list
Part: whole
+------------+-------------------+
| test_name  |        None       |
+------------+-------------------+
| P1 (whole) | 88.86259329931846 |
| P2 (whole) | 61.28795571139848 |
+------------+-------------------+
Protocol #1 Error (MPJPE): 88.86259329931846 mm
Protocol #2 Error (P-MPJPE): 61.28795571139848 mm
----------------------------------------
Loading dataset...
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z
[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_3d...
[overwrite: False] ==> Loading H36M img_2d_canonical_same_z...
[overwrite: False] ==> Loading H36M cam_3d_canonical_same_z...
INFO: Testing
No epoch information in the checkpoint


...

No eval_keypoint. Use part list
Part: whole
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+
| test_name  |     Directions     |    Directions 1    |    Discussion 1    |    Discussion 2    |       Eating       |      Eating 1      |      Greeting      |     Greeting 1    |     Greeting 2     |      Phoning       |     Phon

100%|██████████| 411/411 [02:16<00:00,  3.01it/s]


No eval_keypoint. Use part list
Part: whole
+------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+------------------------+-----------------------+---------------------+----------------------------------+------------------------+--------------------+--------------------+--------------------+---------------------------------+--------------------+-----------------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------

100%|██████████| 323/323 [01:47<00:00,  3.01it/s]


No eval_keypoint. Use part list
Part: whole
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| test_name  |     Directions     |    Di

### Cherrypick e1, e1 -> result_dict.pkl

In [5]:
blacklist_checkpoint = ['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_s15678_tr_54138969_ts_others']

if os.path.exists('result_dict.pkl'):
    print('Loading result_dict.pkl')
    result_dict = readpkl('result_dict.pkl')
else:
    result_dict = {}
result_root = '/home/hrai/codes/MotionBERT/saved_results'
checkpoint_list = os.listdir(result_root)
for checkpoint in tqdm(checkpoint_list):
    if checkpoint in blacklist_checkpoint: continue # skip
    if checkpoint not in result_dict.keys(): result_dict[checkpoint] = {} # create new key if not exists
    checkpoint_root = os.path.join(result_root, checkpoint)
    subset_list = glob(checkpoint_root+'/*.pkl')
    for item in subset_list:
        subset = os.path.basename(item).split('.')[0]
        if subset in result_dict[checkpoint].keys(): 
            #print(f'{subset} exists')
            continue
        result = readpkl(item)
        result_dict[checkpoint][subset] = {'e1': result['e1'], 'e2': result['e2']}
savepkl(result_dict, 'result_dict.pkl')

Loading result_dict.pkl


100%|██████████| 34/34 [00:00<00:00, 25792.43it/s]
